In [3]:
#create glue session
%idle_timeout 30
%timeout 30
%glue_version 5.0
%worker_type G.1X
%number_of_workers 2

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
import pyspark.sql.functions as f
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

You are already connected to a glueetl session 93c698ed-bd3b-4bb8-955e-9ff48b66c808.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is 60 minutes.
idle_timeout has been set to 60 minutes.


You are already connected to a glueetl session 93c698ed-bd3b-4bb8-955e-9ff48b66c808.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current timeout is 60 minutes.
timeout has been set to 60 minutes.


You are already connected to a glueetl session 93c698ed-bd3b-4bb8-955e-9ff48b66c808.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 5.0


You are already connected to a glueetl session 93c698ed-bd3b-4bb8-955e-9ff48b66c808.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: G.1X
Setting new worker type to: G.1X


You are already connected to a glueetl session 93c698ed-bd3b-4bb8-955e-9ff48b66c808.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: 2
Setting new number of workers to: 2



In [12]:
#set bucket directory
transformed_bucket = 'cloud9-transformed'
mart_bucket = 'cloud9-mart'

In [5]:
#load schema from transformed_bucket data catalog
df = glueContext.create_data_frame_from_catalog(database = "cloud9_transformed", table_name = "tr_fire_station")

/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.


In [8]:
# list loaded schema
# df.printSchema()

# print sample rows
# df.show(5)

+-----------------+------------------------+--------------------------------+------------+-----------+-----------+-----------+--------------+----+
|fire_station_name|fire_station_center_name|                         address|       phone|        fax|   latitude|  longitude|employee_count|year|
+-----------------+------------------------+--------------------------------+------------+-----------+-----------+-----------+--------------+----+
| 서울소방재난본부|         개포119안전센터|서울특별시 강남구 논현로 10길...| 02-577-2540|02-529-1119|37.47470048| 127.049895|          NULL|NULL|
| 서울소방재난본부|         삼성119안전센터|서울특별시 강남구 테헤란로 62...|02-6981-7556|02-539-9008|37.51027919|127.0668302|          NULL|NULL|
| 서울소방재난본부|         세곡119안전센터|서울특별시 강남구 밤고개로 26...| 02-451-0882|02-451-0883|37.47040382|127.1072317|          NULL|NULL|
| 서울소방재난본부|         수서119안전센터|서울특별시 강남구 광평로31길 ...| 02-445-9019|02-451-0019|37.48492685|127.0928417|          NULL|NULL|
| 서울소방재난본부|         역삼119안전센터| 서울특별시 강남구 역삼로 236...| 02-557-0109|02-508-119

In [6]:
from pyspark.sql.functions import col
from pyspark.sql.types import *

# 컬럼 필요한 타입으로 변환
df = df.withColumn("year", col("year").cast(IntegerType()))  # "year" 칼럼을 IntegerType으로 변환

df = df \
    .withColumn("fire_station_name", col("fire_station_name")) \
    .withColumn("fire_station_center_name", col("fire_station_center_name")) \
    .withColumn("address", col("address")) \
    .withColumn("phone", col("phone")) \
    .withColumn("fax", col("fax")) \
    .withColumn("latitude", col("latitude")) \
    .withColumn("longitude", col("longitude")) \
    .withColumn("employee_count", col("employee_count")) 

# 결과 확인
df.printSchema()
df.show(3)

root
 |-- fire_station_name: string (nullable = true)
 |-- fire_station_center_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- fax: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- employee_count: long (nullable = true)
 |-- year: integer (nullable = true)

+-----------------+------------------------+--------------------------------+------------+-----------+-----------+-----------+--------------+----+
|fire_station_name|fire_station_center_name|                         address|       phone|        fax|   latitude|  longitude|employee_count|year|
+-----------------+------------------------+--------------------------------+------------+-----------+-----------+-----------+--------------+----+
| 서울소방재난본부|         개포119안전센터|서울특별시 강남구 논현로 10길...| 02-577-2540|02-529-1119|37.47470048| 127.049895|          NULL|NULL|
| 서울소방재난본부|         삼성119안전센터|서울특별시 강남구 테헤란로 62...|02-

In [9]:
# 최근 년도 데이터만 필터하기
from pyspark.sql.functions import max

recent_year = df.agg(max('year')).collect()[0][0]  # 'year' 칼럼의 최대값 구하기

filtered_df = df.filter((col('year') == recent_year) | col('year').isNull())

# 결과 출력
df_mart = filtered_df
#df_mart.show()

+-----------------+------------------------+---------------------------------+------------+------------+-----------+-----------+--------------+----+
|fire_station_name|fire_station_center_name|                          address|       phone|         fax|   latitude|  longitude|employee_count|year|
+-----------------+------------------------+---------------------------------+------------+------------+-----------+-----------+--------------+----+
| 서울소방재난본부|         개포119안전센터| 서울특별시 강남구 논현로 10길...| 02-577-2540| 02-529-1119|37.47470048| 127.049895|          NULL|NULL|
| 서울소방재난본부|         삼성119안전센터| 서울특별시 강남구 테헤란로 62...|02-6981-7556| 02-539-9008|37.51027919|127.0668302|          NULL|NULL|
| 서울소방재난본부|         세곡119안전센터| 서울특별시 강남구 밤고개로 26...| 02-451-0882| 02-451-0883|37.47040382|127.1072317|          NULL|NULL|
| 서울소방재난본부|         수서119안전센터| 서울특별시 강남구 광평로31길 ...| 02-445-9019| 02-451-0019|37.48492685|127.0928417|          NULL|NULL|
| 서울소방재난본부|         역삼119안전센터|  서울특별시 강남구 역삼로 236...| 02-557-

In [26]:
#write mart data to S3
df_mart.write \
              .mode("overwrite") \
              .format('parquet') \
              .partitionBy("fire_station_center_name") \
              .save(f's3://{mart_bucket}/fire_station_mart/')

In [30]:
# Create mt_fire_station table if not exists
spark.sql(
    """
    CREATE EXTERNAL TABLE IF NOT EXISTS `cloud9_mart`.mt_fire_station (
        `fire_station_name` STRING,
        `address` STRING, 
        `phone` STRING, 
        `fax` STRING, 
        `latitude` DOUBLE, 
        `longitude` DOUBLE 
    )
    PARTITIONED BY (`fire_station_center_name` STRING
    )
    ROW FORMAT SERDE 
        'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
    STORED AS INPUTFORMAT 
        'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat' 
    OUTPUTFORMAT 
        'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
    LOCATION
        's3://cloud9-mart/fire_station_mart'
    TBLPROPERTIES (
        'classification' = 'parquet'
    )
    """
)

# Add partition for fire_station_center_name
from datetime import datetime, timedelta

partition_value = "example_center_name"  # Replace with actual partition value

spark.sql(
    f"""
    ALTER TABLE cloud9_mart.mt_fire_station 
    ADD IF NOT EXISTS 
    PARTITION (fire_station_center_name='{partition_value}') 
        LOCATION 's3://cloud9-mart/fire_station_mart/fire_station_center_name={partition_value}/'
    """
)

DataFrame[]
